In [1]:
import sys
print(sys.path)

['', '/opt/openrobots/lib/python3.6/site-packages', '/opt/openrobots/lib/python3.6/dist-packages', '/home/tlha/catkin_crocoddyl_ws/install/lib/python3/dist-packages', '/home/tlha/tiago_public_ws/install/lib/python2.7/dist-packages', '/opt/ros/melodic/lib/python2.7/dist-packages', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/tlha/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/tlha/.local/lib/python3.6/site-packages/IPython/extensions', '/home/tlha/.ipython']


In [2]:
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data

robot = example_robot_data.load('tiago_no_hand')
#robot = example_robot_data.loadTiago(True,'wsg')
robot_model = robot.model
q0 = robot_model.referenceConfigurations["tuck_arm"]
# x0 = np.concatenate([q0, np.zeros(robot_model.nv)])
# robot.q0 = q0
DT = 1e-3
T= 250
target = np.array([3.8, 0.1, 1.1]) 
display = crocoddyl.MeshcatDisplay(robot, 4, 4, False)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
print("referenceConfigurations:", q0)

referenceConfigurations: [ 0.15        0.2        -1.34       -0.2         1.94       -1.57
  1.37        1.          0.          0.          0.70710678  0.70710678
  0.70710678  0.70710678]


In [4]:
print("robot.q0 size:", robot.q0.shape[0])
print(robot.q0)

robot.q0 size: 14
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]


In [5]:
robot_model

Nb joints = 13 (nq=14,nv=12)
  Joint 0 universe: parent=0
  Joint 1 torso_lift_joint: parent=0
  Joint 2 arm_1_joint: parent=1
  Joint 3 arm_2_joint: parent=2
  Joint 4 arm_3_joint: parent=3
  Joint 5 arm_4_joint: parent=4
  Joint 6 arm_5_joint: parent=5
  Joint 7 arm_6_joint: parent=6
  Joint 8 arm_7_joint: parent=7
  Joint 9 head_1_joint: parent=1
  Joint 10 head_2_joint: parent=9
  Joint 11 wheel_left_joint: parent=0
  Joint 12 wheel_right_joint: parent=0

In [6]:
# Create the cost functions
Mref = crocoddyl.FrameTranslation(robot_model.getFrameId("wrist_tool_joint"), target)
state = crocoddyl.StateMultibody(robot.model)
goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, Mref)
xRegCost = crocoddyl.CostModelState(state)
uRegCost = crocoddyl.CostModelControl(state)

In [7]:
# Create cost model per each action model
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)

# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, 1e2)
runningCostModel.addCost("stateReg", xRegCost, 1e-4)
runningCostModel.addCost("ctrlReg", uRegCost, 1e-7)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e5)
terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7)

In [8]:
# Create the actuation model
actuationModel = crocoddyl.ActuationModelFull(state)

# Create the action model
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), DT)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel))


# Create the problem
q0 = robot_model.referenceConfigurations["tuck_arm"]
x0 = np.concatenate([q0, pinocchio.utils.zero(state.nv)])
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

# Solving it with the DDP algorithm
ddp.solve()

# Visualizing the solution in gepetto-viewer
display.displayFromSolver(ddp)

robot_data = robot_model.createData()
xT = ddp.xs[-1]
pinocchio.forwardKinematics(robot_model, robot_data, xT[:state.nq])
pinocchio.updateFramePlacements(robot_model, robot_data)
print('Finally reached = ', robot_data.oMf[robot_model.getFrameId("wrist_tool_joint")].translation.T)

Finally reached =  [0.66177751 0.02910813 1.07980466]


In [9]:
x0.shape[0]

26

In [10]:
display.robot.viewer.jupyter_cell()

In [11]:
xT

array([ 3.75144596e-01,  4.92351353e+00, -3.13248116e+00, -1.54430243e+00,
       -6.03186699e+00, -4.35277582e+00,  8.65263887e-02,  6.00921724e-01,
       -8.06773646e-05, -1.27653996e-01,  7.23836854e-01,  6.89971165e-01,
        7.23836854e-01,  6.89971165e-01, -1.57373116e+00,  4.95910630e+00,
        1.16979357e+00,  2.94255493e+00,  1.10849424e+01,  9.63660213e+00,
       -4.90817197e+00, -1.09821858e+01, -2.82059132e-04,  2.45620085e+00,
       -1.02583673e-03, -1.02583673e-03])